In [1]:
import json
import os
import shutil

import pandas as pd

In [2]:
def parse_result_and_delete_images_and_labels(model, result_file):
    par_dir = os.path.dirname(result_file)
    'num_iter-64_mask_sched-uniform_samp-temp-anneal-False_samp-temp-0.8_eta-0.5'
    T = int(par_dir.split('num_iter-')[-1].split('_')[0])
    mask_sched = par_dir.split('mask_sched-')[-1].split('_')[0]
    samp_temp_anneal = bool(par_dir.split('samp-temp-anneal-')[-1].split('_')[0])
    samp_temp = float(par_dir.split('samp-temp-')[-1].split('_')[0])
    if 'eta' in par_dir:
        eta = float(par_dir.split('eta-')[-1].split('_')[0])
    else:
        eta = 0.0
    with open(result_file, "r") as f:
        result = json.load(f)
        result['Model'] = model.upper()
        result['T'] = T
        result['mask_sched'] = mask_sched
        result['samp_temp_anneal'] = samp_temp_anneal
        result['Temp'] = samp_temp
        result['Eta'] = eta
    
    if os.path.exists(os.path.join(par_dir, "images")):
        shutil.rmtree(os.path.join(par_dir, "images"))
    if os.path.exists(os.path.join(par_dir, "labels")):
        shutil.rmtree(os.path.join(par_dir, "labels"))
    if os.path.exists(os.path.join(par_dir, "images.npz")):
        os.remove(os.path.join(par_dir, "images.npz"))
    return result
    

def highlight_min_max(df, min_cols, max_cols):
    """Highlight the minimum values in min_cols and maximum values in max_cols."""

    def highlight_min(s):
        is_min = s == s.min()
        return ['background-color: lightgreen' if v else '' for v in is_min]

    def highlight_max(s):
        is_max = s == s.max()
        return ['background-color: lightgreen' if v else '' for v in is_max]

    return df.style.apply(highlight_min, subset=min_cols).apply(highlight_max, subset=max_cols)

In [3]:
base_output_dir = "/share/kuleshov/yzs2/maskgit/outputs"
schedule = "uniform"
samp_temp_anneal = False

In [4]:
results = []
for samp_temp in [0.6, 0.8, 1.0]: #, 0.9, 1.0]:
    for n in [16, 32, 64]: #, 128, 256, 512, 1024, 2048]:
        for model in ["mdim_conf", "mdim_const", "mdim_const_guanghan", "mdlm", "maskgit"]:
            result_file = os.path.join(
                base_output_dir,
                model,
                f"num_iter-{n}_mask_sched-{schedule}_samp-temp-anneal-{samp_temp_anneal}_samp-temp-{samp_temp}",
                "results.json")
            if not os.path.exists(result_file):
                for eta in [0.01, 0.02, 0.05]: #, 0.1, 0.2, 0.5]:
                    result_file = os.path.join(
                        base_output_dir,
                        model,
                        f"num_iter-{n}_mask_sched-{schedule}_samp-temp-anneal-{samp_temp_anneal}_samp-temp-{samp_temp}_eta-{eta}",
                        "results.json")
                    if not os.path.exists(result_file):
                        continue
                    results.append(parse_result_and_delete_images_and_labels(model, result_file))
            else:
                results.append(parse_result_and_delete_images_and_labels(model, result_file))
results_df = pd.DataFrame.from_records(results)

In [5]:
df = results_df[['Eta', 'Temp', 'T', 'Model', 'IS', 'FID']]
df['Strategy'] = df.apply(lambda row: row['Model'] + (f" - Eta={row['Eta']}" if row['Eta'] > 0.0 else ''), axis=1)
df = df[['Temp', 'T', 'Strategy', 'IS', 'FID']]
df_styled = highlight_min_max(
    df
    [
        ((df['Temp'] == 0.9) & (df['Strategy'] == 'MDIM_CONF')) |
        ((df['Temp'] == 0.8) & (df['Strategy'] == 'MDIM_CONST - Eta=0.02')) |    
        ((df['Temp'] == 0.8) & (df['Strategy'] == 'MDIM_CONST - Eta=0.05')) |
        ((df['Temp'] == 0.8) & (df['Strategy'] == 'MDLM')) |
        ((df['Temp'] == 1.0) & (df['Strategy'] == 'MASKGIT'))        
     ]
    .groupby(['Strategy', 'Temp', 'T']).first(),
     min_cols=['FID'], max_cols=['IS'])
display(df_styled)

/tmp/ipykernel_3723200/4281375149.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Strategy'] = df.apply(lambda row: row['Model'] + (f" - Eta={row['Eta']}" if row['Eta'] > 0.0 else ''), axis=1)


In [16]:
df = results_df[['Eta', 'Temp', 'T', 'Model', 'IS', 'FID']]
df['Model'] = df['Model'].replace(
    {'MASKGIT': 'MaskGiT', 'MDLM': 'MDLM', 'MDIM_CONST_GUANGHAN': 'ReMDM-const.', 'MDIM_CONST': 'ReMDM-scale', 'MDIM_CONF': 'ReMDM-conf.'}
)
df['Model'] = pd.Categorical(df['Model'], ['MaskGiT', 'MDLM', 'ReMDM-const.', 'ReMDM-scale', 'ReMDM-conf.'])
# df = df.sort_values(['Model', 'Eta', 'Temp', 'T'])
df['Strategy'] = df.apply(lambda row: row['Model'] + (rf" - $\eta={row['Eta']}$" if row['Eta'] > 0.0 else ''), axis=1)
df = df.sort_values(['Model', 'Eta', 'Temp', 'T'])
df['FID'] = df['FID'].astype({'FID': 'float'}).map('{:,.2f}'.format)
df['IS'] = df['IS'].astype({'IS': 'float'}).map('{:,.2f}'.format)
df['Temp'] = df['Temp'].astype({'Temp': 'float'}).map('{:,.1f}'.format)
metric_to_display = 'FID'  # 'FID' or 'IS'
display(df.pivot(index=['Strategy', 'Temp'], columns=['T'], values=['FID', 'IS']))
print(
    df.pivot(
        index=['Strategy', 'Temp'], columns=['T'], values=['FID', 'IS']
    ).to_latex().replace('\cline', '\cmidrule').replace('\multirow[t]{3}', '\multirow[c]{3}')
)

/tmp/ipykernel_3723200/1333942521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Model'] = df['Model'].replace(
/tmp/ipykernel_3723200/1333942521.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Model'] = pd.Categorical(df['Model'], ['MaskGiT', 'MDLM', 'ReMDM-const.', 'ReMDM-scale', 'ReMDM-conf.'])
/tmp/ipykernel_3723200/1333942521.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

FID                    IS                
T                                   16     32     64      16      32      64
Strategy                   Temp                                             
MDLM                       0.6    6.99   7.67   8.43  214.55  233.59  242.28
                           0.8    7.88   5.37   4.69  140.97  169.79  187.93
                           1.0   26.02  18.18  13.96   64.01   82.54   96.30
MaskGiT                    0.6   10.06  11.03  11.72  236.90  244.79  243.64
                           0.8    6.66   7.09   7.75  205.91  225.08  233.87
                           1.0    6.74   4.92   4.85  155.32  181.57  196.38
ReMDM-conf.                0.6    7.46   8.54   9.82  221.25  243.56  251.83
                           0.8    6.91   5.16   5.35  150.73  189.51  212.66
                           1.0   22.14  13.14   8.88   73.00  101.79  126.29
ReMDM-const. - $\eta=0.01$ 0.6    7.07   7.81   8.75  216.00  238.35  243.82
                           0.8    7.71   5.20   4.56  141.31  172.66  194.77
                           1.0   26.25  18.78  14.72   63.97   81.25   93.69
ReMDM-const. - $\eta=0.02$ 0.6    7.10   7.94   9.01  217.12  239.50  244.98
                           0.8    7.55   5.00   4.53  142.33  178.32  198.87
                           1.0   26.86  19.50  15.85   63.06   79.46   91.68
ReMDM-const. - $\eta=0.05$ 0.6    7.21   8.38   9.91  223.96  243.14  244.60
                           0.8    7.24   4.83   4.52  147.20  184.21  211.06
                           1.0   28.11  21.43  19.27   60.47   76.12   82.15
ReMDM-scale - $\eta=0.01$  0.6    7.09   7.78   8.66  215.52  237.82  244.96
                           0.8    7.75   5.24   4.58  141.11  171.56  193.73
                           1.0   26.09  18.48  14.29   64.45   82.26   95.69
ReMDM-scale - $\eta=0.02$  0.6    7.09   7.84   8.90  217.67  239.87  246.37
                           0.8    7.64   5.07   4.48  142.50  176.57  197.35
                           1.0   26.58  18.84  14.78   63.40   81.20   94.99
ReMDM-scale - $\eta=0.05$  0.6    7.19   8.22   9.65  223.05  242.99  250.68
                           0.8    7.40   4.92   4.45  145.27  182.05  209.45
                           1.0   27.39  19.99  16.69   62.09   78.82   90.58

\begin{tabular}{llllllll}
\toprule
 &  & \multicolumn{3}{r}{FID} & \multicolumn{3}{r}{IS} \\
 & T & 16 & 32 & 64 & 16 & 32 & 64 \\
Strategy & Temp &  &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{MDLM} & 0.6 & 6.99 & 7.67 & 8.43 & 214.55 & 233.59 & 242.28 \\
 & 0.8 & 7.88 & 5.37 & 4.69 & 140.97 & 169.79 & 187.93 \\
 & 1.0 & 26.02 & 18.18 & 13.96 & 64.01 & 82.54 & 96.30 \\
\cmidrule{1-8}
\multirow[c]{3}{*}{MaskGiT} & 0.6 & 10.06 & 11.03 & 11.72 & 236.90 & 244.79 & 243.64 \\
 & 0.8 & 6.66 & 7.09 & 7.75 & 205.91 & 225.08 & 233.87 \\
 & 1.0 & 6.74 & 4.92 & 4.85 & 155.32 & 181.57 & 196.38 \\
\cmidrule{1-8}
\multirow[c]{3}{*}{ReMDM-conf.} & 0.6 & 7.46 & 8.54 & 9.82 & 221.25 & 243.56 & 251.83 \\
 & 0.8 & 6.91 & 5.16 & 5.35 & 150.73 & 189.51 & 212.66 \\
 & 1.0 & 22.14 & 13.14 & 8.88 & 73.00 & 101.79 & 126.29 \\
\cmidrule{1-8}
\multirow[c]{3}{*}{ReMDM-const. - $\eta=0.01$} & 0.6 & 7.07 & 7.81 & 8.75 & 216.00 & 238.35 & 243.82 \\
 & 0.8 & 7.71 & 5.20 & 4.56 & 141.31 & 172.66 & 194.77 \\
 & 1.0 &